# PER with Sparse Pauli-Lindblad Model
In quantum simulation algorithms, the result at the end of the algorithm is usually not a single binary value, but instead the expectation value of some operator, which is approximated by averaging over many shots of the same circuit. In the presence of noisy operations, the estimator for such an expectation value is biased, which affects the quality of the result.

The purpose of this notebook is to illustrate a procedure for mitigating this bias and increasing the accuracy of the estimators of the expectation values. 

The first step is performing a tomography experiment to learn the noise model. A process called "Pauli twirling" can be used to convert the noise into a Pauli channel. 

The noise is then assumed to be generated by a master equation with quantum jump operators proportional to Pauli operators. The noise channel can then be expressed as $\Lambda(\rho) = e^{-\mathcal L (\rho)}$, where

$$
\hat{\mathcal L} = \sum_{k}\lambda_k (\hat P_k - \hat I)
$$

This master equation will be referred to as a "sparse Pauli-Lindblad," model. The word sparse indicates that only Pauli operators which have support on physically connected qubits are considered in the model. 

Once the $\lambda_k$ are learned, a partially inverted noise channel can be constructed:

$$
\hat \Lambda^{(\xi)} = \gamma^{(\xi)}\prod_{k}\omega^{(\xi)}_k \hat I+\text{sgn}(1-\xi)(1-\omega_k^{(\xi)})\hat P_k
$$
Where we have defined $\omega^{(\xi)}_k = \frac{1}{2}(1+e^{-2|\xi-1|\lambda_k})$ and
$$
\gamma^{(\xi)}
= \begin{cases}
\exp(2(1-\xi)\sum_k \lambda_k) & \xi < 1 \\
1 & \xi \geq 1
\end{cases}
$$
This channel can be implemented by sampling from it probabilistically and rescaling the results by the overhead $\gamma^{(\xi)}$. It can be shown that as $\xi \to 0$, 
$$
\hat \Lambda^{(\xi)} \to \hat \Lambda^{-1}
$$
the notation has been overloaded slightly, and $\Lambda^{-1}$ is used to indicate the inverse of $\Lambda$. In addition, as $\xi \to \infty$, we have $\Lambda^{(\xi)} \to P_{max}$, where $P_{max}(\rho) = I$ for all $\rho$. Because of this, measurements at different values of $\xi$ can be used to extrapolate to the zero-noise limit with an appropriate fit.

The scope of this notebook is to demonstrate the tools presented here for learning $\lambda_k$ on a simulated noisy backend, and then carrying out PER to approximate a relevant expectation value on a desired circuit.

In [1]:
from qiskit import QuantumCircuit, Aer, transpile
from qiskit.providers.fake_provider import FakeVigoV2
from matplotlib import pyplot as plt
import os
import sys
import numpy as np

sys.path.append(os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "pauli_lindblad_per"))
from tomography.experiment import SparsePauliTomographyExperiment as tomography

plt.style.use("ggplot")

def empty_log(file_path):
    with open(file_path, 'w'):
        pass

def print_log_file(file_path):
    try:
        with open(file_path, 'r') as file:
            for line in file:
                print(line, end='')
    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")
    #empty_log()

# Initialize backend and create circuit
The program requires access to a qiskit backend, using the topology of the processor to find the terms included in the sparse model. The circuit can be any circuit with the requirement that it only contain single and two-qubit gates, and the two-qubit gates must be self-adjoint Clifford gates such as CNOT or CZ gates. Lastly, the circuit should not have any trailing single-qubit gates. This requirement is an edge case that will be removed in future versions.

In [2]:
backend = FakeVigoV2()

In [3]:
qc = QuantumCircuit(2)
qc.rz(np.pi/2,0)
qc.sx(0)
qc.rz(3*np.pi/4,0)

qc.rz(np.pi/2,1)
qc.sx(1)
qc.rz(np.pi/2,1)

qc.cx(0,1)
qc.rz(np.pi/4,0)
qc.cx(0,1)
qc.rz(np.pi/4,0)
qc.cx(0,1)
qc.rz(np.pi/4,0)
qc.cx(0,1)
qc.draw()

┌─────────┐┌────┐┌──────────┐     ┌─────────┐     ┌─────────┐     »
q_0: ┤ Rz(π/2) ├┤ √X ├┤ Rz(3π/4) ├──■──┤ Rz(π/4) ├──■──┤ Rz(π/4) ├──■──»
     ├─────────┤├────┤├─────────┬┘┌─┴─┐└─────────┘┌─┴─┐└─────────┘┌─┴─┐»
q_1: ┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├─┤ X ├───────────┤ X ├───────────┤ X ├»
     └─────────┘└────┘└─────────┘ └───┘           └───┘           └───┘»
«     ┌─────────┐     
«q_0: ┤ Rz(π/4) ├──■──
«     └─────────┘┌─┴─┐
«q_1: ───────────┤ X ├
«                └───┘

## Initialize experiment
The procedure begins by initializing an experiment with the circuits that will later be executed with PER.

In [4]:
empty_log("/home/fabrice/Dokumente/Masterarbeit/AutomatedPERTools/tutorial_notebooks/pauli_lindblad_per/experiment.log")
print_log_file("/home/fabrice/Dokumente/Masterarbeit/AutomatedPERTools/tutorial_notebooks/pauli_lindblad_per/experiment.log")

In [5]:
experiment = tomography(
    circuits = [qc], #list of circuits from which to get the layers requiring tomography
    inst_map = [0,1], #which physical qubits on the processor to map the qubits in the circuit
    backend = backend #quantum backend
    )

In [6]:
print_log_file("/home/fabrice/Dokumente/Masterarbeit/AutomatedPERTools/tutorial_notebooks/pauli_lindblad_per/experiment.log")

2024-07-04 19:59:29,163 Generated layer profile with 1 layers:
2024-07-04 19:59:29,163 ["('cx', (Qubit(QuantumRegister(2, 'q'), 0), Qubit(QuantumRegister(2, 'q'), 1)))"]
2024-07-04 19:59:29,170 [[3, 4], [4, 3], [3, 1], [1, 3], [1, 2], [2, 1], [0, 1], [1, 0]]
2024-07-04 19:59:29,171 <rustworkx.PyDiGraph object at 0x7f59ffde9a80>
2024-07-04 19:59:29,171 Created pauli bases
2024-07-04 19:59:29,171 ['XX', 'YX', 'ZX', 'XY', 'YY', 'ZY', 'XZ', 'YZ', 'ZZ']
2024-07-04 19:59:29,173 Testing here
2024-07-04 19:59:29,173 EdgeList[(0, 1), (1, 0)]
2024-07-04 19:59:29,173 NodeIndices[0, 1]
2024-07-04 19:59:29,173 Created model with the following terms:
2024-07-04 19:59:29,173 {'YZ', 'YI', 'XI', 'IY', 'XY', 'IX', 'IZ', 'YX', 'ZX', 'YY', 'ZI', 'XX', 'XZ', 'ZZ', 'ZY'}
2024-07-04 19:59:29,189 {YZ: YI, YI: YZ, XI: XI, IY: XY, XY: IY, IX: XX, IZ: IZ, YX: ZY, ZX: YY, YY: ZX, ZI: ZZ, XX: IX, XZ: XZ, ZZ: ZI, ZY: YX}
